In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="quick-starts-ws-162706", resource_group='aml-quickstarts-162706', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55')

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-162880
Azure region: southcentralus
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-162880


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name="udacityprojclust"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('cpu cluster already exist. Using it.')
except ComputeTargetException:

    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

cpu cluster already exist. Using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
# Install some dependencies for AutoML to run.

import sys

! {sys.executable} -m pip install azureml-sdk==1.35 azureml-core==1.35.0.post1 azureml-train-automl-runtime==1.35.1 azureml-train==1.35 azureml-train-automl==1.35 azureml-pipeline-core==1.35 azureml-pipeline==1.35 azureml-pipeline-steps==1.35 azureml-widgets==1.35 azureml-opendatasets==1.35 azureml-explain-model==1.35

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps =RandomParameterSampling({'C': uniform(0.5,1.5), 'max_iter': choice(50, 75, 100, 125, 150)})


# Specify a Policy
es_policy = BanditPolicy(slack_factor=0.1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', entry_script='train.py', compute_target=cpu_cluster, script_params={'--C': 'C', '--max_iter':'max_iter'})

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, policy=es_policy, primary_metric_name='Accuracy', max_duration_minutes=100, max_total_runs=10, primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, estimator=est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.core.experiment import Experiment

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [8]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_param = best_run.get_details()['runDefinition']['arguments']

print('Best parameters')
print(best_param)

LR_model = LogisticRegression(C=float(best_param[1]), max_iter=int(best_param[3]))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

LR_model.fit(x_train, y_train)

joblib.dump(LR_model, 'hyperdrive_LR.model')

AttributeError: 'NoneType' object has no attribute 'get_details'

In [ ]:
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core import Datastore


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

data_automl = x
data_automl['y'] = y

datastore = Datastore.get(ws, 'workspaceblobstore')
dataset = TabularDatasetFactory.register_pandas_dataframe(data_automl, target=datastore, name='udacity-project-data')
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target=cluster_name,
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    enable_onnx_compatible_models=True,
    enable_early_stopping=True,
    iterations=10)

In [ ]:
# Submit your automl run

exp_2 = Experiment(ws, 'AutoML_Project')
AutoML_run = exp_2.submit(automl_config, show_output=True)

RunDetails(AutoML_run).show()

In [ ]:
# Retrieve and get insights from your best automl model.

best_run_AutoML, fitted_model_AutoML = AutoML_run.get_output()

In [ ]:
! {sys.executable} -m pip list

In [ ]:
print(fitted_model_AutoML)

In [ ]:
print(hasattr(fitted_model_AutoML, 'steps'))

In [ ]:
type(fitted_model_AutoML)
for attribute in dir(fitted_model_AutoML):
    print(attribute, getattr(fitted_model_AutoML, attribute))

In [ ]:
from pprint import pprint

# Function to list the hyperparameters 

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators' : list(e[0] for e in step[1].estimators), 'weights' : step[1].weights})
            print()

            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        
        else:
            pprint(step[1].get_params())
            print()
        
print_model(fitted_model_AutoML)

In [ ]:
# Get information from guardrails.

print(AutoML_run.get_guardrails())

In [ ]:
# Get information from models.

print(AutoML_run.get_tags())


In [ ]:
# Save the best model by AutoML

joblib.dump(fitted_model_AutoML, 'AutoML.model')

In [ ]:
# Remove compute cluster

cpu_cluster.delete()